Aravind Patnam
Jeremy Tan
Timothy Le

CSE-184 Final Project - Trending Here Trending There 
An analysis on trending and nontrending Youtube videos. 

In [ ]:
## imports
import sys
import pandas as pd
import numpy as np
import random
import requests
import json
import matplotlib.pyplot as plt
import time
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
sns.set_context('notebook')
import warnings
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import re
import random
from gensim import corpora
import gensim
import pickle
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
spacy.load('en')
from spacy.lang.en import English
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import bokeh.plotting as bp
from bokeh.models import HoverTool, ColorBar, NumeralTickFormatter, LinearColorMapper, LassoSelectTool, ResetTool, PanTool, BoxSelectTool, TapTool, PolySelectTool
from bokeh.palettes import plasma
from bokeh.plotting import figure
from bokeh.transform import transform
import bokeh.io
import bokeh.plotting as bpl
import bokeh.models as bmo
from bokeh.palettes import d3
from bokeh.transform import linear_cmap
from bokeh.models import SingleIntervalTicker, LinearAxis
from bokeh.layouts import gridplot
from nltk.stem.wordnet import WordNetLemmatizer
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.palettes import Category20
from bokeh.models import Legend
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
import bokeh
from bokeh.palettes import Category20c, Plasma
from bokeh.models import HoverTool
from bokeh.io import show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import HoverTool, ColorBar, NumeralTickFormatter, LinearColorMapper, LassoSelectTool, ResetTool, PanTool, BoxSelectTool, TapTool, PolySelectTool
# Bokeh Library
from bokeh.io import output_file
from bokeh.models.widgets import Tabs, Panel
warnings.filterwarnings('ignore')

In [ ]:
## written by Aravind Patnam and Jeremy Tan
#### get all csv dataframes for trending. These files should be in the same directory. 
US_trending_df = pd.read_csv('data/USvideos.csv') #USA
CA_trending_df = pd.read_csv('data/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('data/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('data/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('data/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('data/INvideos.csv') #INDIA
JP_trending_df = pd.read_csv('data/JPvideos.csv', encoding='ISO-8859-1') #JAPAN
KR_trending_df = pd.read_csv('data/KRvideos.csv' , encoding='ISO-8859-1') #SOUTH KOREA
MX_trending_df = pd.read_csv('data/MXvideos.csv', encoding='ISO-8859-1') #MEXICO
RU_trending_df = pd.read_csv('data/RUvideos.csv', encoding='ISO-8859-1') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df,
                           JP_trending_df, KR_trending_df, MX_trending_df, RU_trending_df]
list_of_csvs = ['data/USvideos.csv','data/CAvideos.csv', 'data/DEvideos.csv', 'data/FRvideos.csv', 'data/GBvideos.csv', 'data/INvideos.csv', 'data/JPvideos.csv','data/KRvideos.csv', 'data/MXvideos.csv', 'data/RUvideos.csv' ]

big_df = list()
for csv in list_of_csvs:
    # use encoding to bypass utf error
    df = pd.read_csv(csv, index_col='video_id', encoding='ISO-8859-1')
    # add new column called "country" to indentify which videos the csv are coming from
    # depending on your path name, this will break as it looks at the path name 
    df['country'] = csv[5:7]
    big_df.append(df)
    
full_trending_df = pd.concat(big_df)
full_trending_df.tail()

In [ ]:
#### reformatting and detecting nans

# reformat trending_date
full_trending_df['trending_date'] = pd.to_datetime(full_trending_df['trending_date'],errors='coerce', format='%y.%d.%m')
full_trending_df['publish_time'] = pd.to_datetime(full_trending_df['publish_time'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

# detects any nans 
full_trending_df = full_trending_df[full_trending_df['trending_date'].notnull()]
full_trending_df = full_trending_df[full_trending_df['publish_time'].notnull()]

# drop all nans byrmeoving them 
full_trending_df = full_trending_df.dropna(how='any',inplace=False, axis = 0)

# this is done already so don't run it twice
full_trending_df.insert(4, 'publish_date', full_trending_df['publish_time'].dt.date)
full_trending_df['publish_time'] = full_trending_df['publish_time'].dt.time

# set index by video id and sort by trending dates 
full_trending_df_fill = full_trending_df.reset_index().sort_values('trending_date').set_index('video_id')
# set index by vide id and sort by trending dates, but make sure to drop duplicates
full_trending_df = full_trending_df.reset_index().sort_values('trending_date').drop_duplicates('video_id',keep='last').set_index('video_id')
# videos[['publish_date','publish_time']].head()
full_trending_df.head()

In [ ]:
## written by Aravind Patnam

## run these to get the non trending datasets generated from the youtube api. These files should be in the same directory
not_trending_us_df = pd.read_csv('data/not_trending_us_df.csv')
not_trending_ca_df = pd.read_csv('data/not_trending_ca_df.csv')
not_trending_de_df = pd.read_csv('data/not_trending_de_df.csv')
not_trending_fr_df = pd.read_csv('data/not_trending_fr_df.csv')
not_trending_gb_df = pd.read_csv('data/not_trending_gb_df.csv')
not_trending_in_df = pd.read_csv('data/not_trending_in_df.csv')
not_trending_jp_df = pd.read_csv('data/not_trending_jp_df.csv')
not_trending_kr_df = pd.read_csv('data/not_trending_kr_df.csv')
not_trending_mx_df = pd.read_csv('data/not_trending_mx_df.csv')
not_trending_ru_df = pd.read_csv('data/not_trending_ru_df.csv')

The below cell was used to fetch nontrending videos using the youtube API. For each nontrending video, a related non-trending video was fetched. 

In [ ]:
## written by Aravind Patnam

### do not run this unless you have about 1 hour and 10 Youtube API keys and a lot of time!!! Use above already generated datasets for testing. 
f = open("apiKey", "r")
key = f.read()
## for each videoId, find a related video
def do_search_youtube_request(videoId):
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=50&order=relevance&relatedToVideoId={}&type=video&videoDefinition=any&key={}".format(videoId, key)
    r = requests.get(url)
    return r

## given a set of videoIds, find insights (statistics, tags, etc)
def find_video_insights(videoIds):
    print(videoIds)
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={}&key={}'.format(videoIds, key)
    r = requests.get(url)
    return r

## call this with 1 country at a time 
def process_youtube_requests(videoIds):
    df = pd.DataFrame(columns=['video_id', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 
                               'views', 'likes', 'dislikes', 'comment_count', 'description'])
    relatedVideoIds = []
    for videoId in videoIds:
        try: 
            response = do_search_youtube_request(videoId)
            time.sleep(2)
            if (response.status_code == 200):
                r1 = response.json()
                relatedVideoIdItems = r1['items']
                for id in relatedVideoIdItems:
                    relatedVideoId = id['id']['videoId']
                    relatedVideoIds.append(str(relatedVideoId))
            else:
                print(response.status_code)
        except:
            print ("Something went wrong here! 2")
    random.shuffle(relatedVideoIds)
    videoIdsForInsights = []
    for i in range(0, len(relatedVideoIds), 50):
        videoIdsForInsights.append(relatedVideoIds[i:i + n])
    for videoIdList in videoIdsForInsights:
        videoIdsStr = '%2C'.join([str(elem) for elem in videoIdList])
        r2 = find_video_insights(videoIdsStr)
        time.sleep(2)
        if (r2.status_code == 200):
            r = r2.json()
            i = 0
            while (i < len(videoIdList)):
                try:
                    id = videoIdList[i]
                    title = (r['items'][i]['snippet']['title'])
                    channel_title = (r['items'][i]['snippet']['channelTitle'])
                    category_id = (r['items'][i]['snippet']['categoryId'])
                    publish_time = (r['items'][i]['snippet']['publishedAt'])
                    tags = '|'.join((r['items'][i]['snippet']['tags']))
                    views = (r['items'][i]['statistics']['viewCount'])
                    likes = (r['items'][i]['statistics']['likeCount'])
                    dislikes = (r['items'][i]['statistics']['dislikeCount'])
                    comment_count = (r['items'][i]['statistics']['commentCount'])
                    description = (r['items'][i]['snippet']['description'])
                    data = {'video_id': id, 'title': title, 'channel_title': channel_title, 'category_id' : category_id,
                           'publish_time' : publish_time, 'tags' : tags, 'views' : views, 'likes' : likes, 'dislikes' : dislikes,
                           'comment_count' : comment_count, 'description' : description}
                    df = df.append(data, ignore_index = True)
                except:
                    print("Something went wrong! 3")
                i = i + 1
        else:
            print("Something went wrong! 4")
            print (r2.status_code)
            print(r2.text)
    return df     

In [ ]:
## written by Aravind Patnam

### do not run this unless you have a lot of time and 10 Youtube API keys!!!. Datasets are already generated for you for testing. 
n = 50
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()
JP_trending_videoIds = JP_trending_df.sample(n)['video_id'].tolist()
KR_trending_videoIds = KR_trending_df.sample(n)['video_id'].tolist()
MX_trending_videoIds = MX_trending_df.sample(n)['video_id'].tolist()
RU_trending_videoIds = RU_trending_df.sample(n)['video_id'].tolist()

In [ ]:
## written by Aravind Patnam
### do not run this unless you have a lot of time and 10 Youtube API keys!!! Datasets are already generated for you for testing. 
## do following requests separately with a new API Key
not_trending_us_df = process_youtube_requests(US_trending_videoIds)
not_trending_ca_df = process_youtube_requests(CA_trending_videoIds)
not_trending_de_df = process_youtube_requests(DE_trending_videoIds)
not_trending_fr_df = process_youtube_requests(FR_trending_videoIds)
not_trending_gb_df = process_youtube_requests(GB_trending_videoIds)
not_trending_in_df = process_youtube_requests(IN_trending_videoIds)
not_trending_jp_df = process_youtube_requests(JP_trending_videoIds)
not_trending_kr_df = process_youtube_requests(KR_trending_videoIds)
not_trending_mx_df = process_youtube_requests(MX_trending_videoIds)
not_trending_ru_df = process_youtube_requests(RU_trending_videoIds)

In [ ]:
## written by Aravind Patnam

## puts all nontrending and all dfs that we have together for one big df and multiple smaller ones
list_of_all_nontrending_dfs = [not_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gb_df, not_trending_in_df, not_trending_jp_df,
                              not_trending_kr_df, not_trending_mx_df, not_trending_ru_df]


full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)
allDfsList = list_of_all_trending_dfs + list_of_all_nontrending_dfs + [full_trending_df] + [full_nontrending_df]
allDfsDf = pd.concat(allDfsList)
allDfsList.append(allDfsDf)

In [ ]:
## written by Jeremy Tan

## inserts category field into each dataframe from given json file
def insert_category_field(df):
    # add a category column using the category id from the json file
    id_to_category = {}
    with open("data/US_category_id.json", 'r') as f: # the other category json files have missing category ids. We decided to just use the US file since it contained all of them and since they are standard internationally. 
        data = json.load(f)
        for category in data['items']:
            id_to_category[category['id']] = category['snippet']['title']
        categories = []
        for id in list(df['category_id']):
            categories.append(id_to_category[str(id)])
        df.insert(4, 'category', categories)

In [ ]:
## written by Jeremy Tan
## insert new category field into dataframes
for df in allDfsList:
    insert_category_field(df)

In [ ]:
## written by Jeremy Tan

## converts all columns in dataframes to type for analysis
for df in list_of_all_nontrending_dfs:
    df['video_id'] = df['video_id'].astype(str) 
    df['title'] = df['title'].astype(str)
    df['channel_title'] = df['channel_title'].astype(str)
    df['category_id'] = df['category_id'].astype(int)
    df['category'] = df['category'].astype(str)
    df['tags'] = df['tags'].astype(str)
    df['views'] = df['views'].astype(int)
    df['likes'] = df['likes'].astype(int)
    df['dislikes'] = df['dislikes'].astype(int)
    df['comment_count'] = df['comment_count'].astype(int)
    df['description'] = df['description'].astype(str)

In [ ]:
## written by Aravind Patnam

## method to find stats about a given country df
def find_stats(df):
    views = df['views']
    likes = df['likes']
    dislikes = df['dislikes']
    comment_count = df['comment_count']
    tags = list(df['tags'])
    tagsList = []
    for t in tags: 
        tagsList.append(len(t.split('|')))
    viewsStats = [views.sum(), views.sum() / len(views), views.min(), views.max()]
    likesStats = [likes.sum(), likes.sum() / len(likes), likes.min(), likes.max()]
    dislikesStats = [dislikes.sum(), dislikes.sum() / len(dislikes), dislikes.min(), dislikes.max()]
    commentsStats = [comment_count.sum(), comment_count.sum() / len(comment_count), comment_count.min(), comment_count.max()]
    tagsStats = [sum(tagsList), sum(tagsList) / len(tagsList), min(tagsList), max(tagsList)]
    statsDf = pd.DataFrame({'views': viewsStats, 'likes': likesStats, 'dislikes': dislikesStats, 'comment_count': commentsStats, 'tags': tagsStats}, index = ['count', 'mean', 'min', 'max']) 
    return statsDf

In [ ]:
## written by Aravind Patnam

## output is a map containing all the numeric data for each df
describes = []
for df in allDfsList:
    describes.append(find_stats(df))
describesKeys = ['US_trending_df', 'CA_trending_df', 'DE_trending_df', 'FR_trending_df', 'GB_trending_df',
                'IN_trending_df', 'JP_trending_df', 'KR_trending_df', 'MX_trending_df', 'RU_trending_df',
                 'not_trending_us_df', 'not_trending_ca_df', 'not_trending_de_df', 'not_trending_fr_df',
                'not_trending_gb_df', 'not_trending_in_df', 'not_trending_jp_df', 'not_trending_kr_df',
                 'not_trending_mx_df', 'not_trending_ru_df',
                'full_trending_df', 'full_nontrending_df', 'allDfsDf']
describeMap = dict(zip(describesKeys, describes))
describeMap

In [ ]:
## written by Aravind Patnam

## make sense of each data point we have from the describeMap and clean it for visualization
countries = list(describeMap.keys())
views_count = []
views_mean = []
views_max = []
views_min = []
likes_count = []
likes_mean = []
likes_max = []
likes_min = []
dislikes_count = []
dislikes_mean = []
dislikes_max = []
dislikes_min = []

comments_mean = []
comments_max = []
comments_min = []

tags_mean = []
tags_max = []
tags_min = []

for a in countries:
    views_count.append(describeMap[a].loc['count', 'views'])
    views_mean.append(describeMap[a].loc['mean', 'views'])
    views_max.append(describeMap[a].loc['max', 'views'])
    views_min.append(describeMap[a].loc['min', 'views'])
    
    likes_count.append(describeMap[a].loc['count', 'likes'])
    likes_mean.append(describeMap[a].loc['mean', 'likes'])
    likes_max.append(describeMap[a].loc['max', 'likes'])
    likes_min.append(describeMap[a].loc['min', 'likes'])
    
    dislikes_count.append(describeMap[a].loc['count', 'dislikes'])
    dislikes_mean.append(describeMap[a].loc['mean', 'dislikes'])
    dislikes_max.append(describeMap[a].loc['max', 'dislikes'])
    dislikes_min.append(describeMap[a].loc['min', 'dislikes'])
    
    comments_mean.append(describeMap[a].loc['mean', 'comment_count'])
    comments_max.append(describeMap[a].loc['max', 'comment_count'])
    comments_min.append(describeMap[a].loc['min', 'comment_count'])
    
    tags_mean.append(describeMap[a].loc['mean', 'tags'])
    tags_max.append(describeMap[a].loc['max', 'tags'])
    tags_min.append(describeMap[a].loc['min', 'tags'])

Hover over the bar graphs to play with them and see what each one represents!! To show the scale of these numbers better, the graph only goes till 10000. A numeric analysis amongst likes, views, dislikes, comments, and tags is presented. 

In [ ]:
## written by Aravind Patnam

# visualize above numeric data
bokeh.io.reset_output()
bokeh.io.output_notebook()



countries = describesKeys
categories = ['views_mean', 'views_max', 'views_min']
categoriesLikes = ['likes_mean', 'likes_max', 'likes_min']
categoriesDislikes = ['dislikes_mean', 'dislikes_max', 'dislikes_min']
categoriesComments = ['comments_mean', 'comments_max', 'comments_min']
categoriesTags = ['tags_mean', 'tags_max', 'tags_min']

data = {'countries' : countries,
        'views_mean': views_mean,
        'views_max':views_max, 
        'views_min':views_min }
dataLikes = {'countries' : countries, 'likes_mean' : likes_mean, 'likes_max': likes_max, 'likes_min' : likes_min}

dataDislikes = {'countries' : countries, 'dislikes_mean' : dislikes_mean, 'dislikes_max' : dislikes_max, 'dislikes_min': dislikes_min}

dataComments = {'countries' : countries, 'comments_mean' : comments_mean, 'comments_max' : comments_max, 'comments_min': comments_min}

dataTags = {'countries' : countries, 'tags_mean' : tags_mean, 'tags_max': tags_max, 'tags_min' : tags_min}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (c, category) for c in countries for category in categories ]
# print (len(x))
yLikes = [(c, l) for c in countries for l in categoriesLikes]
yDislikes = [(c, h) for c in countries for h in categoriesDislikes]

yComments = [(c, comms) for c in countries for comms in categoriesComments]

yTags = [(c, ts) for c in countries for ts in categoriesTags]

counts = sum(zip(data['views_mean'], data['views_max'], data['views_min']), ()) # like an hstack
countsLikes = sum(zip(dataLikes['likes_mean'], dataLikes['likes_max'], dataLikes['likes_min']), ()) # like an hstack
countsDislikes = sum(zip(dataDislikes['dislikes_mean'], dataDislikes['dislikes_max'], dataDislikes['dislikes_min']), ())
countsComments = sum(zip(dataComments['comments_mean'], dataComments['comments_max'], dataComments['comments_min']), ())

countsTags = sum(zip(dataTags['tags_mean'], dataTags['tags_max'], dataTags['tags_min']), ())




source = ColumnDataSource(data=dict(x=x, counts=counts))
sourceLikes = ColumnDataSource(data=dict(x=yLikes, counts=countsLikes))
sourceDislikes = ColumnDataSource(data=dict(x=yDislikes, counts=countsDislikes))

sourceComments = ColumnDataSource(data=dict(x=yComments, counts=countsComments))

sourceTags = ColumnDataSource(data=dict(x=yTags, counts=countsTags))


hover = HoverTool()

hover.tooltips = [
    ("(x,y)", "($x, $y)"),
    ("Country", "@x"),
    ("Stat" , "@counts")
]

p = figure(x_range=FactorRange(*x), plot_height=250, plot_width=3000, title="Views Counts Per Country Trending/NonTrending",
           toolbar_location=None, tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()])

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral6, factors=categories, start=1, end=2))


p1 = figure(x_range=FactorRange(*yLikes), plot_height=250, plot_width=3000, title="Likes Counts Per Country Trending/NonTrending",
           toolbar_location=None, tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()])

p1.vbar(x='x', top='counts', width=0.9, source=sourceLikes, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral6, factors=categoriesLikes, start=1, end=2))


p2 = figure(x_range=FactorRange(*yDislikes), plot_height=250, plot_width=3000, title="Dislikes Counts Per Country Trending/NonTrending",
           toolbar_location=None, tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()])

p2.vbar(x='x', top='counts', width=0.9, source=sourceDislikes, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral6, factors=categoriesDislikes, start=1, end=2))



p3 = figure(x_range=FactorRange(*yComments), plot_height=250, plot_width=3000, title="Comment Counts Per Country Trending/NonTrending",
           toolbar_location=None, tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()])

p3.vbar(x='x', top='counts', width=0.9, source=sourceComments, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral6, factors=categoriesComments, start=1, end=2))


p4 = figure(x_range=FactorRange(*yTags), plot_height=250, plot_width=3000, title="Tags Counts Per Country Trending/NonTrending",
           toolbar_location=None, tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()])

p4.vbar(x='x', top='counts', width=0.9, source=sourceTags, line_color="white",
       fill_color=factor_cmap('x', palette=Spectral6, factors=categoriesTags, start=1, end=2))

p.y_range.start = 0
p.y_range.end = 10000
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

p1.y_range.start = 0
p1.y_range.end = 10000
p1.x_range.range_padding = 0.1
p1.xaxis.major_label_orientation = 1
p1.xgrid.grid_line_color = None

p2.y_range.start = 0
p2.y_range.end = 10000
p2.x_range.range_padding = 0.1
p2.xaxis.major_label_orientation = 1
p2.xgrid.grid_line_color = None


p3.y_range.start = 0
p3.y_range.end = 10000
p3.x_range.range_padding = 0.1
p3.xaxis.major_label_orientation = 1
p3.xgrid.grid_line_color = None

p4.y_range.start = 0
p4.y_range.end = 100
p4.x_range.range_padding = 0.1
p4.xaxis.major_label_orientation = 1
p4.xgrid.grid_line_color = None


# Bokeh Library
from bokeh.io import output_file
from bokeh.models.widgets import Tabs, Panel



# Create two panels, one for each conference
pPanel = Panel(child=p, title='Views Analysis')
p1Panel = Panel(child=p1, title='Likes Analysis')
p2Panel = Panel(child=p2, title='Dislikes Analysis')
p3Panel = Panel(child=p3, title='Comments Analysis')
p4Panel = Panel(child=p4, title='Tags Analysis')

# Assign the panels to Tabs
tabs = Tabs(tabs=[pPanel, p1Panel, p2Panel, p3Panel, p4Panel])

# Show the tabbed layout
show(tabs)


Hover over the bar graphs to play with them and see what each one represents!! Each one shows the likes-dislikes ratio amongst trending and non-trending videos.

In [ ]:
## written by Aravind Patnam

## visualizes likes to dislikes ratio from above numeric data

likesToDislikesRatio = [i / j for i, j in zip(likes_count, dislikes_count)] 
# Bokeh libraries


bokeh.io.reset_output()
bokeh.io.output_notebook()
colorlist = Category20c[13] + Plasma[10]

hover = HoverTool()

hover.tooltips = [
    ("Country", "@countries"),
    ("Stat" , "@likesToDislikesRatio")
]

source = ColumnDataSource(data=dict(countries=countries, likesToDislikesRatio=likesToDislikesRatio))
pRatio = figure(x_range=countries, plot_height=250, plot_width=2500, toolbar_location=None, title="Likes To Dislikes", tools = [hover])
pRatio.vbar(x='countries', top='likesToDislikesRatio', width=0.9, source=source, 
           line_color='white', fill_color=factor_cmap('countries', palette=colorlist, factors=countries))

pRatio.xgrid.grid_line_color = None
pRatio.y_range.start = 0
pRatio.y_range.end = 50
pRatio.legend.orientation = "horizontal"
pRatio.legend.location = "top_center"

show(pRatio)

In [ ]:
## written by Aravind Patnam

###get most common tags from a country df
def get_most_common_tags(country_df):
    tags = country_df['tags'].to_string(index=False, header=False)
    split_tags = [i.replace('"', '') for i in tags.split("|")]
    stop_words = stopwords.words('english')
    filtered_tags = [word for word in split_tags if word not in stop_words]
    fdist = FreqDist(split_tags)
    most_popular_tags = fdist.most_common(1000)
    return dict(most_popular_tags)

In [ ]:
## written by Aravind Patnam

## a bunch of maps containing the most popular tags for each country that are both trending and not trending and their frequency
us_trending_most_common_tags = get_most_common_tags(US_trending_df)
ca_trending_most_common_tags = get_most_common_tags(CA_trending_df)
de_trending_most_common_tags = get_most_common_tags(DE_trending_df)
fr_trending_most_common_tags = get_most_common_tags(FR_trending_df)
gb_trending_most_common_tags = get_most_common_tags(GB_trending_df)
in_trending_most_common_tags = get_most_common_tags(IN_trending_df)
jp_trending_most_common_tags = get_most_common_tags(JP_trending_df)
kr_trending_most_common_tags = get_most_common_tags(KR_trending_df)
mx_trending_most_common_tags = get_most_common_tags(MX_trending_df)
ru_trending_most_common_tags = get_most_common_tags(RU_trending_df)

us_nontrending_most_common_tags = get_most_common_tags(not_trending_us_df)
ca_nontrending_most_common_tags = get_most_common_tags(not_trending_ca_df)
de_nontrending_most_common_tags = get_most_common_tags(not_trending_de_df)
fr_nontrending_most_common_tags = get_most_common_tags(not_trending_fr_df)
gb_nontrending_most_common_tags = get_most_common_tags(not_trending_gb_df)
in_nontrending_most_common_tags = get_most_common_tags(not_trending_in_df)
jp_nontrending_most_common_tags = get_most_common_tags(not_trending_jp_df)
kr_nontrending_most_common_tags = get_most_common_tags(not_trending_kr_df)
mx_nontrending_most_common_tags = get_most_common_tags(not_trending_mx_df)
ru_nontrending_most_common_tags = get_most_common_tags(not_trending_ru_df)

Below is a wordcloud visualization of some of the countries and a comparison of them between trending and non-trending videos. Since there is not much space to do all 20 different combinations, only the 3 with the most popular tags were displayed. 

In [ ]:
## written by Aravind Patnam


## wordcloud visualization on countries with most top tags

links = ['https://cdn.pixabay.com/photo/2017/03/14/21/00/american-flag-2144392_960_720.png', 
        'https://i.pinimg.com/originals/93/85/dd/9385dde0f8cf96e0c60e5e659036b303.jpg',
        'https://cdn.britannica.com/97/1597-004-7C2918C6/Flag-India.jpg']

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(20, 20))
mask_USA = np.array(Image.open(requests.get(links[0], stream=True).raw))
mask_GB = np.array(Image.open(requests.get(links[1], stream=True).raw))
mask_IN = np.array(Image.open(requests.get(links[2], stream=True).raw))
image_colors_usa = ImageColorGenerator(mask_USA)
image_colors_gb = ImageColorGenerator(mask_GB)
image_colors_in = ImageColorGenerator(mask_IN)
wordcloud_usa_trending = WordCloud(width=1000, height=1000, mask=mask_USA, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(us_trending_most_common_tags.keys())))
wordcloud_gb_trending = WordCloud(width=1000, height=1000, mask=mask_GB, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(gb_trending_most_common_tags.keys())))
wordcloud_in_trending = WordCloud(width=1000, height=1000, mask=mask_IN, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(in_trending_most_common_tags.keys())))


wordcloud_usa_nontrending = WordCloud(width=1000, height=1000, mask=mask_USA, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(us_nontrending_most_common_tags.keys())))
wordcloud_gb_nontrending = WordCloud(width=1000, height=1000, mask=mask_GB, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(gb_nontrending_most_common_tags.keys())))
wordcloud_in_nontrending = WordCloud(width=1000, height=1000, mask=mask_IN, background_color="white", max_words=1000, max_font_size=1000).generate(str(list(in_nontrending_most_common_tags.keys())))


plt.figure(figsize=(20,20), facecolor='k')

axes[0, 0].imshow(wordcloud_usa_trending.recolor(color_func=image_colors_usa), interpolation="bilinear")
axes[0, 0].set_title("USA Trending Tags")
axes[0, 0].axis("off")

axes[1, 0].imshow(wordcloud_gb_trending.recolor(color_func=image_colors_gb), interpolation="bilinear")
axes[1, 0].set_title("Great Britain Trending Tags")
axes[1, 0].axis("off")


axes[2, 0].imshow(wordcloud_in_trending.recolor(color_func=image_colors_in), interpolation="bilinear")
axes[2, 0].set_title("India Trending Tags")
axes[2, 0].axis("off")

axes[0, 1].imshow(wordcloud_usa_nontrending.recolor(color_func=image_colors_usa), interpolation="bilinear")
axes[0, 1].set_title("USA Non-Trending Tags")
axes[0, 1].axis("off")


axes[1, 1].imshow(wordcloud_gb_nontrending.recolor(color_func=image_colors_gb), interpolation="bilinear")
axes[1, 1].set_title("Great Britain Non-Trending Tags")
axes[1, 1].axis("off")




axes[2, 1].imshow(wordcloud_in_nontrending.recolor(color_func=image_colors_in), interpolation="bilinear")
axes[2, 1].set_title("India Non-Trending Tags")
axes[2, 1].axis("off")



plt.tight_layout(pad=0)
plt.show()

Below represents a sentiment analysis conducted on the youtube tags. A datset of positive words and negative words is provided for the model to work. 

In [ ]:
## written by Aravind Patnam

## do sentiment analysis on each of the tags
## return classifications on each of the tags in both trending and non-trending per country
positive_words_df = pd.read_fwf('data/positivewords.txt')
negative_words_df = pd.read_fwf('data/negativewords.txt')

def extract_features(words):
    return dict([(word, True) for word in words.split()])

def build_sentiment_analysis_model():
    positive_words = positive_words_df['positivewords'].values.tolist()
    negative_words = negative_words_df['negativewords'].values.tolist()
    pos_feats = [(extract_features(f), 'positive') for f in positive_words ]
    neg_feats = [(extract_features(f), 'negative') for f in negative_words ]
    dataset = pos_feats + neg_feats
    random.shuffle(dataset)
    cutoff = int(0.80 * len(dataset))
    train_data = dataset[:cutoff]
    test_data = dataset[cutoff:]

    classifier = NaiveBayesClassifier.train(train_data)
    print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    return classifier

def execute_model(tags):
    classifications = {}
    classifier = build_sentiment_analysis_model()
    # classifier.show_most_informative_features(5)
    for tag in tags:
        classified = classifier.classify(extract_features(tag))
        classifications[tag] = classified
    return classifications

This is the accuracy with which the model runs. 

In [ ]:
## written by Aravind Patnam

## gets the classifications from the sentiment analysis and prints out accuracies of the model
classifications_us_trending = execute_model(list(us_trending_most_common_tags.keys()))
classifications_ca_trending = execute_model(list(ca_trending_most_common_tags.keys()))
classifications_de_trending = execute_model(list(de_trending_most_common_tags.keys()))
classifications_fr_trending = execute_model(list(fr_trending_most_common_tags.keys()))
classifications_gb_trending = execute_model(list(gb_trending_most_common_tags.keys()))
classifications_in_trending = execute_model(list(in_trending_most_common_tags.keys()))
classifications_jp_trending = execute_model(list(jp_trending_most_common_tags.keys()))
classifications_kr_trending = execute_model(list(kr_trending_most_common_tags.keys()))
classifications_mx_trending = execute_model(list(mx_trending_most_common_tags.keys()))
classifications_ru_trending = execute_model(list(ru_trending_most_common_tags.keys()))

classifications_us_nontrending = execute_model(list(us_nontrending_most_common_tags.keys()))
classifications_ca_nontrending = execute_model(list(ca_nontrending_most_common_tags.keys()))
classifications_de_nontrending = execute_model(list(de_nontrending_most_common_tags.keys()))
classifications_fr_nontrending = execute_model(list(fr_nontrending_most_common_tags.keys()))
classifications_gb_nontrending = execute_model(list(gb_nontrending_most_common_tags.keys()))
classifications_in_nontrending = execute_model(list(in_nontrending_most_common_tags.keys()))
classifications_jp_nontrending = execute_model(list(jp_nontrending_most_common_tags.keys()))
classifications_kr_nontrending = execute_model(list(kr_nontrending_most_common_tags.keys()))
classifications_mx_nontrending = execute_model(list(mx_nontrending_most_common_tags.keys()))
classifications_ru_nontrending = execute_model(list(ru_nontrending_most_common_tags.keys()))

In [ ]:
## written by Aravind Patnam

## pulls actual statistics from classifications for visualization
def get_sentiment_stats(classification, country):
    sentiments = list(classification.values())
    sentiments_df = pd.DataFrame(sentiments, columns=['Sentiment'])
    negatives = len(sentiments_df[sentiments_df['Sentiment'] =='negative'])
    positives = len(sentiments_df[sentiments_df['Sentiment'] =='positive'])
    total_len = len(sentiments_df)
    percentage_of_negative = negatives / total_len * 100
    percentage_of_positive = positives / total_len * 100
    ratio = positives/negatives
    # ratioStr = "{} positive/negative ratio: {} -----> {}% positives of total , {}% negatives of total".format(country, ratio, percentage_of_positive, percentage_of_negative)
    # print (ratioStr)
    return country, positives, negatives

In [ ]:
## written by Aravind Patnam

## calls above method for stats for each country for visualization

country_us_trending, us_trending_pos, us_trending_neg = get_sentiment_stats(classifications_us_trending, "USA_Trending")
country_ca_trending, ca_trending_pos, ca_trending_neg = get_sentiment_stats(classifications_ca_trending, "Canada_Trending")
country_de_trending, de_trending_pos, de_trending_neg = get_sentiment_stats(classifications_de_trending, "Denmark_Trending")
country_fr_trending, fr_trending_pos, fr_trending_neg = get_sentiment_stats(classifications_fr_trending, "France_Trending")
country_gb_trending, gb_trending_pos, gb_trending_neg = get_sentiment_stats(classifications_gb_trending, "GreatBritain_Trending")
country_in_trending, in_trending_pos, in_trending_neg = get_sentiment_stats(classifications_in_trending, "India_Trending")
country_jp_trending, jp_trending_pos, jp_trending_neg = get_sentiment_stats(classifications_jp_trending, "Japan_Trending")
country_kr_trending, kr_trending_pos, kr_trending_neg = get_sentiment_stats(classifications_kr_trending, "SouthKorea_Trending")
country_mx_trending, mx_trending_pos, mx_trending_neg = get_sentiment_stats(classifications_mx_trending, "Mexico_Trending")
country_ru_trending, ru_trending_pos, ru_trending_neg = get_sentiment_stats(classifications_ru_trending, "Russia_Trending")


country_us_nontrending, us_nontrending_pos, us_nontrending_neg = get_sentiment_stats(classifications_us_nontrending, "USA_NonTrending")
country_ca_nontrending, ca_nontrending_pos, ca_nontrending_neg = get_sentiment_stats(classifications_ca_nontrending, "Canada_NonTrending")
country_de_nontrending, de_nontrending_pos, de_nontrending_neg = get_sentiment_stats(classifications_de_nontrending, "Denmark_NonTrending")
country_fr_nontrending, fr_nontrending_pos, fr_nontrending_neg = get_sentiment_stats(classifications_fr_nontrending, "France_NonTrending")
country_gb_nontrending, gb_nontrending_pos, gb_nontrending_neg = get_sentiment_stats(classifications_gb_nontrending, "GreatBritain_NonTrending")
country_in_nontrending, in_nontrending_pos, in_nontrending_neg = get_sentiment_stats(classifications_in_nontrending, "India_NonTrending")
country_jp_nontrending, jp_nontrending_pos, jp_nontrending_neg = get_sentiment_stats(classifications_jp_nontrending, "Japan_NonTrending")
country_kr_nontrending, kr_nontrending_pos, kr_nontrending_neg = get_sentiment_stats(classifications_kr_nontrending, "SouthKorea_NonTrending")
country_mx_nontrending, mx_nontrending_pos, mx_nontrending_neg = get_sentiment_stats(classifications_mx_nontrending, "Mexico_NonTrending")
country_ru_nontrending, ru_nontrending_pos, ru_nontrending_neg = get_sentiment_stats(classifications_ru_nontrending, "Russia_NonTrending")

countries = [country_us_trending, country_ca_trending, country_de_trending, country_fr_trending, country_gb_trending,
            country_in_trending, country_jp_trending, country_kr_trending, country_mx_trending, country_ru_trending,
            country_us_nontrending, country_ca_nontrending, country_de_nontrending, country_fr_nontrending, 
            country_gb_nontrending, country_in_nontrending, country_jp_nontrending, country_kr_nontrending, 
            country_mx_nontrending, country_ru_nontrending]
positivePercenteages = [us_trending_pos, ca_trending_pos, de_trending_pos, fr_trending_pos, gb_trending_pos, in_trending_pos, jp_trending_pos,
            kr_trending_pos, mx_trending_pos, ru_trending_pos, us_nontrending_pos, ca_nontrending_pos, de_nontrending_pos, fr_nontrending_pos,
            gb_nontrending_pos, in_nontrending_pos, jp_nontrending_pos, kr_nontrending_pos, mx_nontrending_pos, ru_nontrending_pos]
negativePercentages = [us_trending_neg, ca_trending_neg, de_trending_neg, fr_trending_neg, gb_trending_neg, in_trending_neg, jp_trending_neg, 
                      kr_trending_neg, mx_trending_neg, ru_trending_neg, us_nontrending_neg, ca_nontrending_neg, de_nontrending_neg, 
                      fr_nontrending_neg, gb_nontrending_neg, in_nontrending_neg, jp_nontrending_neg, kr_nontrending_neg, mx_nontrending_neg,
                      ru_nontrending_neg]
posToNegRatios = [i / j for i, j in zip(positivePercenteages, negativePercentages)] 
data_dict = {"Country" : countries, "Positives": positivePercenteages, "Negatives": negativePercentages, "PositiveNegativeRatio": posToNegRatios}
data = pd.DataFrame(data_dict, columns = ['Country', 'Positives' , 'Negatives', 'PositiveNegativeRatio'])

data

Hover over the plots and switch table to play with them and see what each one represents!! Use the toolbar to crop out some plots for better analysis. The first plot visualizes the ratios between the positive and negative tags while the second one shows the actual values that were classified as positive and negative by the sentiment analysis classifier. 

In [ ]:
## written by Aravind Patnam

## visualizes sentiment analysis ratios

## import pandas as pd


bokeh.io.reset_output()
bokeh.io.output_notebook()
mySource = bp.ColumnDataSource(data)
#Use the field name of the column source
mapper = linear_cmap(field_name='Positives', palette="Spectral6" ,low=max(list(data['Positives']) + list(data['Negatives'])) ,high=min(list(data['Positives']) + list(data['Negatives'])))

hover = HoverTool()
hover.tooltips = [
    ("Country", "@Country"),
    ("Ratio" , "@PositiveNegativeRatio"),
    ("Positives", "@Positives"),
    ("Negatives" , "@Negatives")
]
myPlot = bp.figure(
               title = 'Positives vs. Negatives',
               plot_height=500,
               plot_width=500, 
    tools = [BoxSelectTool(), LassoSelectTool(), ResetTool(), hover, TapTool(), PolySelectTool()], 
    background_fill_color = "black",
    x_axis_label = 'Positives', y_axis_label = 'Negatives',
x_range=(0, 1000), y_range=(0, 1000))
myPlot.title.text_font_size = '20pt'
myPlot.xaxis.axis_label_text_font_size = "20pt"
myPlot.yaxis.axis_label_text_font_size = "20pt"
myPlot.xaxis.ticker = SingleIntervalTicker(interval=50)
myPlot.yaxis.ticker = SingleIntervalTicker(interval=50)
myPlot.circle("Positives",
          "Negatives",
              line_color=mapper,color=mapper, fill_alpha=1, 
          source=mySource,
          size = 12,
             selection_color='deepskyblue',
           nonselection_color='lightgray',
             hover_fill_color='yellow', hover_alpha=2.0)
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))

myPlot.add_layout(color_bar, 'right')



bokeh.io.reset_output()
bokeh.io.output_notebook()


countries = list(data['Country'])
hover = HoverTool(
    tooltips=[
        ("Ratio", "@top")
    ]
)

colors = ['red', 'red', 'red', 'red',
          'red', 'red', 'red', 'red',
          'red', 'red', 'green', 'green', 'green', 'green',
          'green', 'green', 'green', 'green',
          'green', 'green']


p = figure(x_range=countries, plot_height=500, title="Sentiment Analysis Ratios", width = 500,  background_fill_color = "black")
legends = ['Trending', 'Not Trending']
v = p.vbar(x=countries, top=list(data['PositiveNegativeRatio']), width=0.5, color=colors)
v1 = p.vbar(x=countries, top=list(data['PositiveNegativeRatio']), width=0.5, color='green')
p.add_layout(Legend(items=[
        ("Trending"   , [v]),
        ("NonTrending" , [v1]),
    ]))
v1.visible = False

p.add_tools(hover)
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = -1.5
p.xaxis.axis_label = "Countries"
p.yaxis.axis_label = "Ratio of Positive:Negative"
p.xgrid.grid_line_color = None

p.title.text_font_size = '20pt'
p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"




# Increase the plot widths
myPlot.plot_width = p.plot_width = 1000

# Create two panels, one for each conference
myPlotPanel = Panel(child=myPlot, title='Positives vs. Negatives')
pPanel = Panel(child=p, title='Sentiment Analysis Ratios')

# Assign the panels to Tabs
tabs = Tabs(tabs=[pPanel, myPlotPanel])

# Show the tabbed layout
show(tabs)

In [ ]:
## written by Aravind Patnam


## lda model prep for topic model inference from youtube tags and description accross trending and nontrending videos

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(str(text))
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
## written by Aravind Patnam

## method to execute LDA model and clean the LDA input one more time
def do_LDA(lda_input):
    text_data = []
    for line in lda_input:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            text_data.append(tokens)
    topics, corpus, dictionary = execute_LDA(text_data)
    return topics, corpus, dictionary
    
def execute_LDA(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open('data/corpus.pkl', 'wb'))
    dictionary.save('data/dictionary.gensim')
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
    ldamodel.save('data/model5.gensim')
    topics = ldamodel.print_topics(num_words=10)
    return topics, corpus, dictionary

def clean_lda_input(input):
    l = []
    for a in input:
        text = re.sub(r"http\S+", "", str(a))
        l.append(text)
    return l

In [ ]:
## written by Aravind Patnam

## visualize LDA using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings

def visualize_LDA(start, corpus, dictionary):
    if (start == True):
        lda = gensim.models.ldamodel.LdaModel.load('data/model5.gensim')
        lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
        return lda_display

Hover and play around with the principle component analysis presented below. LDA topic model was used on youtube tags and descriptions to show these findings. The relevancy metric at the time of this submission is set to around 0.30 since it showed the most favorable results that were not too specific and not too generic. 

In [ ]:
## written by Aravind Patnam

## this will take a long time to run!

## visualize trending videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings

full_trending_lda_input = list(get_most_common_tags(full_trending_df).keys()) + list (full_trending_df.sample(16901)['description'])
topics_Full_Trending, corpus, dictionary = do_LDA(full_trending_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam

## this will take a long time to run!

## visualize nontrending videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings


full_nontrending_lda_input = list(get_most_common_tags(full_nontrending_df).keys()) + list (full_nontrending_df['description'])
topics_Full_Nontrending, corpus, dictionary = do_LDA(full_nontrending_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam

## this will take a long time to run!

## visualize all videos using pyLDAvis -> this might be only visible on nbviewer depending on your notebook viewing settings

allDfsDf_lda_input = list(get_most_common_tags(allDfsDf).keys()) + list (allDfsDf.sample(100000)['description'])
topics_all_dfs, corpus, dictionary = do_LDA(allDfsDf_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [ ]:
## written by Aravind Patnam
## no need to run this as this has already been done for you
## convert to csvs

not_trending_us_df.to_csv('data/not_trending_us_df.csv')
not_trending_ca_df.to_csv('data/not_trending_ca_df.csv')
not_trending_de_df.to_csv('data/not_trending_de_df.csv')
not_trending_fr_df.to_csv('data/not_trending_fr_df.csv')
not_trending_gb_df.to_csv('data/not_trending_gb_df.csv')
not_trending_in_df.to_csv('data/not_trending_in_df.csv')
not_trending_jp_df.to_csv('data/not_trending_jp_df.csv')
not_trending_kr_df.to_csv('data/not_trending_kr_df.csv')
not_trending_mx_df.to_csv('data/not_trending_mx_df.csv')
not_trending_ru_df.to_csv('data/not_trending_ru_df.csv')

In [ ]:
# written by Jeremy Tan 
# How long did a video stay trending?
new = pd.DataFrame(full_trending_df_fill.groupby([full_trending_df_fill.index,'country']).count()['title'].sort_values(ascending=False)).reset_index()
new.head(), new.tail()
video_list,max_list = list(),list()
country_list = full_trending_df.groupby(['country']).count().index

bokeh.io.reset_output()
bokeh.io.output_notebook()

# gabs countries and grabs the times it appears in the dataframe
# print(country_list)
for c in country_list:
    video_list.append(new[new['country']==c]['title'].value_counts().sort_index())
    max_list.append(max(new[new['country']==c]['title'].value_counts().sort_index().index))
# print(video_list) 
# print(max_list)

# make line plots for the different countries (repeat another 9 times)
first_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[0],
             toolbar_location=None)

# plot for different countries (repeat another 9 times)
first_fig.line(x=video_list[0].index, y=video_list[0], 
         color='gray', line_width=1)

second_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[1],
             toolbar_location=None)

second_fig.line(x=video_list[1].index, y=video_list[1], 
         color='gray', line_width=1)

third_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[2],
             toolbar_location=None)

third_fig.line(x=video_list[2].index, y=video_list[2], 
         color='gray', line_width=1)


fourth_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[3],
             toolbar_location=None)

fourth_fig.line(x=video_list[3].index, y=video_list[3], 
         color='gray', line_width=1)

fifth_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[4],
             toolbar_location=None)

fifth_fig.line(x=video_list[4].index, y=video_list[4], 
         color='gray', line_width=1)

sixth_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[5],
             toolbar_location=None)

sixth_fig.line(x=video_list[5].index, y=video_list[5], 
         color='gray', line_width=1)


seventh_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[6],
             toolbar_location=None)

seventh_fig.line(x=video_list[6].index, y=video_list[6], 
         color='gray', line_width=1)

eight_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[7],
             toolbar_location=None)

eight_fig.line(x=video_list[7].index, y=video_list[7], 
         color='gray', line_width=1)

ninth_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[8],
             toolbar_location=None)

ninth_fig.line(x=video_list[8].index, y=video_list[8], 
         color='gray', line_width=1)

tenth_fig = figure(x_axis_type='linear',
             plot_height=300, plot_width=600,
             title='How long did a video stay trending?',
             x_axis_label='Conccurent Appearences (days)', y_axis_label=country_list[9],
             toolbar_location=None)

tenth_fig.line(x=video_list[9].index, y=video_list[9], 
         color='gray', line_width=1)

#Add panels for tabs 
first_panel = Panel(child=first_fig, title='Canada')
second_panel = Panel(child=second_fig, title='Germany')
third_panel = Panel(child=third_fig, title='France')
fourth_panel = Panel(child=fourth_fig, title='Great Britan')
fifth_panel = Panel(child=fifth_fig, title='India')
sixth_panel = Panel(child=sixth_fig, title='Japan')
seventh_panel = Panel(child=seventh_fig, title='Korea')
eighth_panel = Panel(child=eight_fig, title='Mexico')
ninth_panel = Panel(child=ninth_fig, title='Russia')
tenth_panel = Panel(child=tenth_fig, title='United States')


# Assign the panels to Tabs
tabs = Tabs(tabs=[first_panel, second_panel, third_panel, fourth_panel, fifth_panel, sixth_panel, seventh_panel, eighth_panel, ninth_panel, tenth_panel])
show(tabs)

As you can see from the visualization above, most trending videos only stay trending for two days or less. Some notable outliers are videos from Great Britan, which has videos that will stay trending for close to 30 days! For countries in the far East, videos only seem to stay tredning for one day so the turnover rate is pretty hight

In [ ]:
# Written by Jeremy Tan 
# Makes two new columns that count tilte length and description length
full_trending_df['title_length'] = full_trending_df['title'].str.len()
full_trending_df['description_length'] = full_trending_df['description'].str.len()

In [ ]:
# Written by Jeremy Tan
# What is the correlation between views, likes, disklies, and comment count 
full_trending_df = full_trending_df.rename(columns={'comment_count':'comments'})
# make a scatter matrix to visualzie correlations
fig = px.scatter_matrix(full_trending_df, dimensions=['views', 'likes', 'dislikes', 'comments', 'title_length', 'description_length'])
fig.update_traces(opacity=0.5, showupperhalf=False)
fig.update_layout(title = "Correlations", width = 900, height = 900)
fig.show()

As seen from the scatter matrix, there is a somewhat strong correlation between views and likes. Another correlation is between likes and comments, views and comments, and dislikes and comments (though somewhat weak!). Another interesting point is the length of the description and the length of the title seem to be tied together.

In [ ]:
# What is the correlation between views, likes, disklies, and comment count?
# Another interactive visualization 
corr = full_trending_df.loc[:, ['views', 'likes', 'dislikes', 'comments', 'description_length', 'title_length']].corr()
fig2 = go.Figure(data=go.Heatmap(
        z=corr.values,
        x=corr.index,
        y=corr.index,
        colorscale="Blues",
        zmin=-1,
        zmax=1
))
fig2.update_layout(title='Correlations between title, description, comments, dislikes, likes, and views')
fig2.show()

Another way to visualzie the correlations. Here, I can see additional correlations of dislikes and views, dislikes and likes, and dislikes and comments.

In [ ]:
# Written by Jeremy Tan
# What is the correlation between views, likes, disklies, and comment count in categories?
categories = full_trending_df['category'].unique()
interactions_corr_list = [full_trending_df[full_trending_df['category'] == cat].loc[:, ['views', 'likes', 'dislikes', 'comments', 'description_length', 'title_length']].corr() for cat in categories]
fig3 = go.Figure()
for idx, corr in enumerate(interactions_corr_list):
    fig3.add_trace(
        go.Heatmap(
            z=corr.values,
            x=corr.index,
            y=corr.index,
            colorscale="Blues",
            zmin=-1,
            zmax=1,
            visible=False)) 

# Add buttons
fig3.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list([
                dict(label=cat,
                     method="update",
                     args=[{"visible": [False if sub_idx != idx else True for sub_idx, sub_cat in enumerate(categories)]},
                           {"title": "Correlation heatmap for category: " + cat}])
                for idx, cat in enumerate(categories)
            ] ) 
        )
    ])

Click a category you want to see the correlation of!

Another way to visualize correlations but now based on categories. For the popular categories, the correlations I stated earlier hold true. However, if you were to go to a unpopular category like "trailers," negative correlations appear in place of the previous, strong correlations. Most suprisingly, for categories that elict human emotion, such as "Pet & Animals" and "Nonprofits & Activism" there is a strong correlation between likes, views, comments, and dislikes. 

In [ ]:
# Written by Jeremy Tan
# What category of videos trend the most in which countries?
# What category of videos fail to hit trending in which countries?
def trendingCategories(country, name):
    cat = country['category'].value_counts().reset_index()
    pop = px.bar(x=cat['category'], y=cat['index'], orientation='h')
    pop.update_layout(title_text="Most successful categories of trend videos in " + name, height=400)
    pop.update_xaxes(title_text="Number of Videos")
    pop.update_yaxes(title_text="Categories")
    pop.show()
    
def nontrendingCategories(country, name):
    cat = country['category'].value_counts().reset_index()
    pop = px.bar(x=cat['category'], y=cat['index'], orientation='h')
    pop.update_layout(title_text="Categories that fail to trend in " + name, height=400)
    pop.update_xaxes(title_text="Number of Videos")
    pop.update_yaxes(title_text="Categories")
    pop.show()

In [ ]:
trendingCategories(US_trending_df, "United States")

In [ ]:
nontrendingCategories(not_trending_us_df, "United States")

Based on the two plot above, one can see "Entertainment" videos are the majority of videos made. They, however, have the highest chance to both fail and succeed. The pattern illuminates the more successful the category, the more videos that are being put out.  

In [ ]:
trendingCategories(CA_trending_df, "Canada")

In [ ]:
nontrendingCategories(not_trending_ca_df, "Canada")

The same pattern seems to appear as it did in the United States. However, "People and Blogs" and "Music" seem to fail more in this country.

In [ ]:
trendingCategories(DE_trending_df, "Germany")

In [ ]:
nontrendingCategories(not_trending_de_df, "Germany")

The same pattern seems to appear as it did in the United States. However, "People and Blogs" have more of a success rate but "Films & Animation" seem to fail more.

In [ ]:
trendingCategories(FR_trending_df, "France")

In [ ]:
nontrendingCategories(not_trending_fr_df, "France")

Same pattern occurs like in the Untied States. An interesting point is that "Peoples & Blog" have a higher probability of being trending. We can assume this is due to people valuing these type of videos more.

In [ ]:
trendingCategories(GB_trending_df, "Great Britan")

In [ ]:
nontrendingCategories(not_trending_gb_df, "Great Britan")

Same pattern occurs like in the Untied States. However, "Music" dominates this country with the majority of the videos being produced are of this category.

In [ ]:
trendingCategories(IN_trending_df, "India")

In [ ]:
nontrendingCategories(not_trending_in_df, "India")

Same pattern as the US. Nothing interesitng to note as it follows the pattern exactly where the more successful a category, the more likely the video will also fail.

In [ ]:
trendingCategories(JP_trending_df, "Japan")

In [ ]:
nontrendingCategories(not_trending_jp_df, "Japan")

Similar pattern as Germany, where "People and Blogs" videos will have a higher chance to be tredning and not fail.

In [ ]:
trendingCategories(KR_trending_df, "Korea")

In [ ]:
nontrendingCategories(not_trending_kr_df, "Korea")

Same pattern as the US.Interesting enough, this country cares alot about "News & Politics" and have lots of video that hit trending (only second behind "Entertainment"). 

In [ ]:
trendingCategories(MX_trending_df, "Mexico")

In [ ]:
nontrendingCategories(not_trending_mx_df, "Mexico")

Same pattern as the US. Similar stats as the US too. 

In [ ]:
trendingCategories(RU_trending_df, "Russia")

In [ ]:
nontrendingCategories(not_trending_ru_df, "Russia")

Same pattern as the US. However, like Germany and Japan, "Peoples & Blogs" have a high succes rate to reach trending and fail less.

In [ ]:
# Written by Jeremy Tan
# Which categories are the most popular? Do highest average amongst likes and views ----> shows which type of videos people enjoy the most
# makes a box plot to visuzlaize distbution of likes and views 
def likes_to_categories(country, name, log):
    country['likes_log'] = np.log(country['likes'] + 1)
    country['views_log'] = np.log(country['views'] + 1)
    country['dislikes_log'] = np.log(country['dislikes'] + 1)
    country['comment_log'] = np.log(country['comment_count'] + 1)
    if "likes" in log:
        fig = px.box(country, y=log, x='category')
        fig.update_layout(title_text="Like Distribuition by Category in " + name, height=700)
        fig.update_yaxes(title_text="Likes(log)")
        fig.show()
    else:
        fig = px.box(country, y=log, x='category')
        fig.update_layout(title_text="View Distribuition by Category in " + name, height=700)
        fig.update_yaxes(title_text="Views(log)")
        fig.show()
    

In [ ]:
likes_to_categories(US_trending_df, "United States", 'likes_log')

In [ ]:
likes_to_categories(US_trending_df, "United States", 'views_log')

Judging from the boxplot, it seems people in the US enjoy videos categorized Music more the most. Gaming is the second most popualr. Then, Entertainment. 

In [ ]:
likes_to_categories(CA_trending_df, "Canada", 'likes_log')

In [ ]:
likes_to_categories(CA_trending_df, "Canada", 'views_log')

It isn't as clear, but there seems to be a tie between Movies and Music as some of the more popular videos in Canada. Comedy lags behind third. 

In [ ]:
likes_to_categories(DE_trending_df, "Germany", 'likes_log')

In [ ]:
likes_to_categories(DE_trending_df, "Germany", 'views_log')

In Germany, Music is the most popular category. Movies trail right behind, and Comedy right after.

In [ ]:
likes_to_categories(FR_trending_df, "France", 'likes_log')

In [ ]:
likes_to_categories(FR_trending_df, "France", 'views_log')

In France, Music is the most popular category like Germany. However, Comedy and Entertainment trail right behind.

In [ ]:
likes_to_categories(GB_trending_df, "Great Britan", 'likes_log')

In [ ]:
likes_to_categories(GB_trending_df, "Great Britan", 'views_log')

Music is the most popular category, while Nonprfits & Activism then Entertainment trail behind.

In [ ]:
likes_to_categories(IN_trending_df, "India", 'likes_log')

In [ ]:
likes_to_categories(IN_trending_df, "India", 'views_log')

It isn't clear which videos come out on top, but based on likes, Pets & Animals, Gaming, and then Comedy are the top categories.

In [ ]:
likes_to_categories(JP_trending_df, "Japan", 'likes_log')

In [ ]:
likes_to_categories(JP_trending_df, "Japan", 'views_log')

Funny enough, Science & Technology are the most popular videos follwoed by Music and then Comedy.

In [ ]:
likes_to_categories(KR_trending_df, "Korea", 'likes_log')

In [ ]:
likes_to_categories(KR_trending_df, "Korea", 'views_log')

Similar to Japan, Science & Technology is first as Musis is second. However, Sports is third. 

In [ ]:
likes_to_categories(MX_trending_df, "Mexico", 'likes_log')

In [ ]:
likes_to_categories(MX_trending_df, "Mexico", 'views_log')

Music is first, Gaming is second, and Comedy is third.

In [ ]:
likes_to_categories(RU_trending_df, "Russia", 'likes_log')

In [ ]:
likes_to_categories(RU_trending_df, "Russia", 'views_log')

Music is first. Science and Technology is second. Comedy is third. 

In [ ]:
# Written by Jeremy Tan
# Which channels are the most successfucl at reaching trending?
# Grabs most reoccuring videos in trending 
def videos_top(country, name):
  
    tmp = country.channel_title.value_counts()[:25]    
   
    pop = px.bar(x=tmp, y=tmp.index, orientation='h')
    pop.update_layout(title_text="Top Channels", height=700)
    pop.update_xaxes(title_text="# times channel reached trending")
    pop.update_yaxes(title_text="channel name")

    pop.show()

In [ ]:
videos_top(US_trending_df, "United States")

In [ ]:
videos_top(CA_trending_df, "Canada")

In [ ]:
videos_top(DE_trending_df, "Germany")

In [ ]:
videos_top(FR_trending_df, "France")

In [ ]:
videos_top(GB_trending_df, "Great Britan")

In [ ]:
videos_top(IN_trending_df, "India")

In [ ]:
videos_top(JP_trending_df, "Japan")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
videos_top(KR_trending_df, "Korea")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
videos_top(MX_trending_df, "Mexico")

In [ ]:
videos_top(RU_trending_df, "Russia")

Original scrapper did not properly encode characters correctly. Hence the weird symbols.

In [ ]:
# Written by Jeremy Tan
# prep data to by adding like rate and spliitng publish time into a hour, min, and sec column
full_trending_df['like_rate'] =  full_trending_df ['likes'] / full_trending_df['views'] * 100
full_trending_df[['hour','min','sec']] = full_trending_df['publish_time'].astype(str).str.split(':', expand=True).astype(int)
full_trending_df.head()

In [ ]:
# Written by Jeremy Tan
# When a video gets published, what is the intial like rate that got them to trending?
# Has two plots: one shows which hour a video is most commonly published and the other is a boxen plot that shows differnet quartiles of the like_rate + outliers


fig = make_subplots(
    rows=2, cols=1, subplot_titles=("Hours Videos are Published", "Like Rate")
)


s = full_trending_df['hour'].value_counts() 
new = pd.DataFrame({'Hour':s.index, 'Count':s.values}) 

fig.add_trace(go.Bar(x=new['Hour'], y=new['Count']), row=1, col=1)
fig.add_trace(go.Box(x=full_trending_df["hour"], y=full_trending_df["like_rate"]), row=2, col=1)
fig.update_xaxes(title_text="Hour", row=1, col=1)
fig.update_xaxes(title_text="Hour", row=2, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Like Rate(log)", row=2, col=1)


fig.show()



It seems most trending videos are published at 4 pm. However, more engagemnt in terms of like happens at 7pm. The graphs make the notebook lag, but hiding one of the traces helps!

In [ ]:
full_nontrending_df.head()

In [ ]:
# Written by Jeremy Tan 
# Grabs the videos that have the most views, likes, or dislikes
# What videos have the most views, likes, and dislikes in the tredning dataset and nontrending dataset?
def visualize_most(my_df, column): 
    # sort df by column
    sorted_df = my_df.sort_values(column, ascending=False).iloc[:10]
    # make a bar plot
    pop = px.bar(x=sorted_df['title'], y=sorted_df[column])
    pop.update_layout(title_text="Top Ten " + column, height=700)
    pop.update_yaxes(title_text="#")
    pop.update_xaxes(title_text="Video Title")
    pop.show()

In [ ]:
visualize_most(full_trending_df, "views")

In [ ]:
visualize_most(full_nontrending_df, "views")

In [ ]:
visualize_most(full_trending_df, "likes")

In [ ]:
visualize_most(full_nontrending_df, "likes")

In [ ]:
visualize_most(full_trending_df, "dislikes")

In [ ]:
visualize_most(full_nontrending_df, "dislikes")

In [ ]:
# Written by Jeremy Tan
# Which country has the most active participation and engagemet?
country_list = full_trending_df.groupby(['country']).count().index
p = ['views', 'likes', 'dislikes', 'comments']
measures = list()
for i, typ in enumerate(p):
    measure = list()
    for c in country_list:
        measure.append(full_trending_df[full_trending_df['country']==c][typ].agg('sum')/len(full_trending_df[full_trending_df['country']==c].index.unique()))
    measures.append(measure)
    
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Views", "Likes", "Dislikes", "Comments")
)

# Add traces
fig.add_trace(go.Bar(x=list(country_list), y=measures[0]), row=1, col=1)
fig.add_trace(go.Bar(x=list(country_list), y=measures[1]), row=1, col=2)
fig.add_trace(go.Bar(x=list(country_list), y=measures[2]), row=2, col=1)
fig.add_trace(go.Bar(x=list(country_list), y=measures[3]), row=2, col=2)
fig.update_layout(title_text="Viewer Engagement", height=700)

fig.update_xaxes(title_text="countries", row=1, col=1)
fig.update_xaxes(title_text="countries", row=1, col=2)
fig.update_xaxes(title_text="countries",  row=2, col=1)
fig.update_xaxes(title_text="countries", row=2, col=2)

fig.update_yaxes(title_text="num", row=1, col=1)
fig.update_yaxes(title_text="num", row=1, col=2)
fig.update_yaxes(title_text="num", row=2, col=1)
fig.update_yaxes(title_text="num", row=2, col=2)

fig.show()



Overall, Great Britan has the most active audience with the US seriously lagging behind.